In [1]:
%matplotlib widget
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
import h5py, time, datetime, json
import scipy.integrate as integrate
from scipy.optimize import curve_fit, brentq
from lmfit import minimize, Parameters, report_fit
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
# Format axes for publication
def format_ax(ax, oneside=True):
    ax.tick_params(axis="x", which='major', direction='in')
    ax.tick_params(axis="x", which='minor', direction='in')
    ax.tick_params(axis="y", which='major', direction='in')
    ax.tick_params(axis="y", which='minor', direction='in')
    ax.xaxis.set_ticks_position('both')
    if oneside:
        ax.yaxis.set_ticks_position('both')

from numpy import genfromtxt
data = genfromtxt('data/HISTOGRAM_32x01_C0_317uK.dat', delimiter='  ')
C0_g0_01 = data[:,0]
C0_pr_01 = data[:,1]

data = genfromtxt('data/HISTOGRAM_32x01_CG_311uK.dat', delimiter='  ')
Cg_g0_01 = data[:,0]
Cg_pr_01 = data[:,1]

data = genfromtxt('data/HISTOGRAM_32x20_C0_317uK.dat', delimiter='  ')
C0_g0_20 = data[:,0]
C0_pr_20 = data[:,1]

data = genfromtxt('data/HISTOGRAM_32x20_CG_311uK.dat', delimiter='  ')
Cg_g0_20 = data[:,0]
Cg_pr_20 = data[:,1]

data = genfromtxt('data/PHYSICAL_CG.dat', delimiter='  ')
calphysCg_Te = data[:,0]
calphysCg_g0 = data[:,1]
calphysCg_3s = data[:,2]*3
calphysCg_dT = calphysCg_3s*np.abs(np.gradient(calphysCg_Te, calphysCg_g0))

data = genfromtxt('data/RANDOM_CG.dat', delimiter='  ')
calrandCg_Te = data[:,0]
argsort = np.argsort(calrandCg_Te)
calrandCg_Te = calrandCg_Te[argsort]
calrandCg_g0 = data[:,1][argsort]
calrandCg_3s = data[:,2][argsort]*3
calrandCg_dT = calrandCg_3s*np.abs(np.gradient(calrandCg_Te, calrandCg_g0))

data = genfromtxt('data/MASTER_EQUATION_CALIBRATION_CBT.dat', delimiter='  ')
me_Te = data[:,0]
me_g0 = data[:,1]

data = genfromtxt('data/EC_SCALE.dat', delimiter='  ')
scale_C0 = data[:,0]
scale_Ec = data[:,1]

Cg=1.82 #pF
C0=0.19 #pF
Cs = Cg + 2*C0

In [2]:
fig = plt.figure(figsize=(5, 8))
outer_gs = gridspec.GridSpec(2, 1, height_ratios = [3, 6], hspace=0.2)
gs1 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer_gs[0])
gs2 = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec = outer_gs[1], hspace=0, height_ratios = [5, 2])
ax=[]
ax.append(plt.subplot(gs1[0]))
ax.append(plt.subplot(gs2[0]))
ax.append(plt.subplot(gs2[1]))
ax.append(fig.add_axes([0.595, 0.21, 0.29, 0.21], zorder=3))
#fig.tight_layout()

################# TOP PLOT #################
mean_Cg_g0_20 = np.sum(Cg_g0_20*Cg_pr_20)/np.sum(Cg_pr_20)
mean_C0_g0_20 = np.sum(C0_g0_20*C0_pr_20)/np.sum(C0_pr_20)

mean_Cg_g0_01 = np.sum(Cg_g0_01*Cg_pr_01)/np.sum(Cg_pr_01)
mean_C0_g0_01 = np.sum(C0_g0_01*C0_pr_01)/np.sum(C0_pr_01)

lineCg20, = ax[0].plot(Cg_g0_20, Cg_pr_20, color=colors[0])
lineC020, = ax[0].plot(C0_g0_20, C0_pr_20, color=colors[1])
lineCg01, = ax[0].plot(Cg_g0_01, Cg_pr_01, '--', color=colors[0], alpha=0.5)
lineC001, = ax[0].plot(C0_g0_01, C0_pr_01, '--', color=colors[1], alpha=0.5)

ax[0].plot([mean_Cg_g0_01, mean_Cg_g0_01], [np.min(Cg_pr_01), np.max(Cg_pr_20)], '--k') # , label=f'$\mu$={mean_Cg_g0_01:.3f}'

ax[0].text(0.05, 0.8, 'N=33\nMCMC Random', transform=ax[0].transAxes)
ax[0].text(0.65, 0.85, r'Constant $\mathrm{C_\Sigma}$', transform=ax[0].transAxes)
ax[0].set_xlabel(r'$\mathrm{g_0/g_T}$')
ax[0].set_ylabel(r'Probability density')
ax[0].grid(which='both')
ax[0].set_xlim(0.442, 0.472)
ax[0].set_xticks([0.45, 0.46, 0.47])
ax[0].set_yticks([0])
ax[0].xaxis.set_label_coords(0.455, -0.08)
ax[0].legend([lineCg20, lineCg01, lineC020, lineC001 ], ['M=20', 'M=1', '', ''], ncol=2, markerfirst=False, handletextpad=0.7, framealpha=1, edgecolor='w', loc=(0.49,0.35))
ax[0].text(0.66, 0.62, 'gCBT\n$\mathrm{C_g \gg C_j}$', ha='center', color=colors[0], transform=ax[0].transAxes)
ax[0].text(0.84, 0.62, 'jCBT\n$\mathrm{C_j \gg C_g}$', ha='center', color=colors[1], transform=ax[0].transAxes)
#, title=r"           $\mathrm{C_g \gg C_0}$   $\mathrm{C_0 \gg C_g}$"
rect = patches.Rectangle((0.55,0.55), 0.38, 0.30, linewidth=1, edgecolor='w', facecolor='w', transform=ax[0].transAxes, zorder=2)
ax[0].patches.append(rect)

############### BOTTOM PLOT #################
ax[1].plot(calrandCg_Te[calrandCg_Te<600e-3]*1e3, calrandCg_g0[calrandCg_Te<600e-3], label='MCMC Random')
ax[1].plot(calphysCg_Te[calphysCg_Te<600e-3]*1e3, calphysCg_g0[calphysCg_Te<600e-3], '-.', color=colors[0], label='MCMC Free Energy')

ax[1].fill_between(calrandCg_Te*1e3, calrandCg_g0-calrandCg_3s, calrandCg_g0+calrandCg_3s, alpha=0.5, color=colors[0], linewidth=0)

ax[1].plot(me_Te[me_Te<600e-3], me_g0[me_Te<600e-3], '--', label='Master Equation', color=colors[0])
ax[1].set_yticks(np.linspace(0,1,6))
#ax[1].set_ylim(-0.05, 0.9)
ax[1].text(0.105, 0.47, "M=20")

calphysCg_g0_interp = np.interp(calrandCg_Te, calphysCg_Te, calphysCg_g0)
calphysCg_diffg0_interp_plus = np.abs((calrandCg_g0+calrandCg_3s) - calphysCg_g0_interp)
calphysCg_diffg0_interp_minus = np.abs((calrandCg_g0-calrandCg_3s) - calphysCg_g0_interp)
calphysCg_diffg0_interp = np.maximum(calphysCg_diffg0_interp_plus, calphysCg_diffg0_interp_minus)
calphysCg_diffg0_interp = np.maximum(calphysCg_diffg0_interp, calrandCg_3s)
calphysCg_dT_interp = calphysCg_diffg0_interp*np.abs(np.gradient(calrandCg_Te, calrandCg_g0))

err_percent_randCg = 100*calrandCg_dT/calrandCg_Te
err_percent_physCg = 100*calphysCg_dT_interp/calrandCg_Te
ax[2].plot(calrandCg_Te*1e3, err_percent_randCg, color=colors[0], label='Rand.')
#ax[2].plot(calrandCg_Te*1e3, err_percent_physCg, '-.', color=colors[0])

upperTe = np.interp(calrandCg_g0+calrandCg_3s, calrandCg_g0, calrandCg_Te)
lowerTe = np.interp(np.interp(calrandCg_Te, calphysCg_Te, calphysCg_g0), calrandCg_g0, calrandCg_Te)
ax[2].plot(calrandCg_Te*1e3, 100*(upperTe-lowerTe)/calrandCg_Te, color=colors[5], label='Cons.')

lintemp = np.linspace(0.1, 0.15, 101)
#err_interp = np.interp(lintemp, calrandCg_Te*1e3, err_percent_randCg)
#minTemp = np.min(lintemp[err_interp<10])

xticks = np.array([0.1, 0.3, 0.737, 1])
xticklabels = np.array(['100', '300', r'$\mathrm{E_c/k_B}$', '1000'])

for i in range(2):
    ax[i+1].set_xlim(0.07,2.2)
    ax[i+1].set_xscale('log')
    ax[i+1].set_xticks(xticks)
    ax[i+1].grid()
    
ax[1].set_xticklabels([])
ax[2].set_xticklabels(xticklabels)
ax[2].xaxis.get_majorticklabels()[3].set_horizontalalignment("left")
ax[1].set_ylabel(r'$\mathrm{g_0/g_T}$')
ax[2].set_xlabel(r'$\mathrm{T_e (μK)}$')

ax[1].legend(loc=2, framealpha=1, edgecolor='w')
ax[2].legend(loc=(0.29, 0.49), framealpha=1, edgecolor='w')

ax[2].set_ylim(0.1,280)
ax[2].set_yscale('log')
ax[2].set_yticks([1, 3, 10, 30, 100])
ax[2].set_yticklabels([1, 3, 10, 30, 100])
ax[2].set_ylabel(r'$\mathrm{\Delta T/T (\%)}$')

############### INSET PLOT #################
rect = patches.Rectangle((0.525,-0.25), 0.47, 0.80, linewidth=1, edgecolor='w', facecolor='w', zorder=2, transform=ax[1].transAxes, alpha=0.9)
fig.patches.append(rect)
ax[3].plot(scale_C0, scale_Ec, 'k',  zorder=15)
ax[3].plot([0.001,1], [0.5,0.5], '--k')
ax[3].set_ylim(0.45,1.02)
ax[3].set_xticks([0, 2*C0/Cs, 0.5, 1])
ax[3].set_xticklabels(["0", f"{2*C0/Cs:.2f}", "0.5", "1"])
ax[3].set_yticks([0.5, 1.0])
ax[3].set_xlabel(r'$\mathrm{2C_j/C_\Sigma}$')
ax[3].xaxis.set_label_coords(0.5, -0.15)
ax[3].yaxis.set_label_coords(-0.075, 0.5)
ax[3].set_ylabel(r'$\mathrm{E_c(e^2/C_\Sigma)}$')
#ax[3].annotate(r'$\mathrm{\frac{N-1}{N}}$', xy=(1,32/33), xytext=(0.5,0.93),
#               arrowprops=dict(edgecolor=colors[1], arrowstyle='->'))
ax[3].fill_between([0,0.4], [0.5,0.5], [1.5,1.5], color=colors[0], alpha=0.1, linewidth=0)
ax[3].fill_between([0.9,1], [0.5,0.5], [1.5,1.5], color=colors[1], alpha=0.1, linewidth=0)
ax[3].text(0.04, 0.95, 'gCBT', ha='left', color=colors[0])
ax[3].text(0.98, 0.95, 'jCBT', ha='right', color=colors[1])

ax[3].annotate("This\nwork", xy=(2*C0/Cs,0.5), xytext=(0.05,0.70), arrowprops=dict(edgecolor=colors[0], arrowstyle='->'))
for i in range(4):
    format_ax(ax[i])
for i in range(2):
    ax[i].text(-0.12, 1, f'({"abcd"[i]})', transform=ax[i].transAxes)

fig.savefig('F3.pdf', transparent=True, bbox_inches='tight', pad_inches=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …